## 6.5 卷积神经网络迁移学习
### 6.5.1 迁移学习介绍
在6.4节中介绍了1998年提出的LeNet5模型和2015年提出的Inception-v3模型。对别二者可以发现CNN的层数和复杂度都发生了巨大的变化，下表给出了2012-2015年ILSVRC第一名模型的层数（在这里层数只计算了卷积层和全连接层的个数，没有参数的池化层没有包括在内）及top5的错误率。
<p align='center'>
    <img src=images/表6.2.JPG>
</p>

从上表可以看到，随着模型层数及复杂度的增加，模型在ImageNet上的错误率也随之降低。然而，训练、复杂的卷积神经网络需要非常多的标注数据。如6.1节中提到的，ImageNet图像分类数据集中有120万标注图片，所以才能将152层的ResNet的模型训练到大约96.5% 的正确率。在其实的应用中，很难收集到如此多的标注数据。即使可以收集到，也需要花费大量人力物力。而且即使有海量的训练数据，要训练一个复杂的卷积神经网络也需要几天甚至几周的时间。为了解决标注数据和训练时间的问题，可以使用本节将要介绍的迁移学习。

**所谓迁移学习，就是将一个问题上训练好的模型通过简单的调整使其适用于一个新的问题**。本节将介绍如何利用ImageNet数据集上训练好的Inception-v3 模型来解决一个新的图像分类问题。根据论文[DeCAF: A Deep Convolutional Activation Feature for Generic Visual Recognition](http://proceedings.mlr.press/v32/donahue14.pdf)中的结论：`可以保留训练好Inception-v3模型中所有卷积层的参数，只是替换最后一层全连接层`。在最后这一层全连接层之前的网络层称之为瓶颈层（bottleneck）。

- 原理：**将新的图像通过训练好的卷积神经网络直到瓶颈层的过程可以看成是对图像进行特征提取的过程**。在训练好的Inception-v3模型中，因为将瓶颈层的输出再通过一个单层的全连接层神经网络可以很好地区分1000种类别的图像，所以有理由认为**瓶颈层输出的节点向量可以被作为任何图像的一个更加精简且表达能力更强的特征向量。**于是，在新数据集上，可以直接利用这个训练好的神经网络对图像进行特征提取，然后再将提取得到的特征向量作为输入来训练一个新的单层全连接神经网络处理新的分类问题。

- 适用条件：一般来说，在数据量足够的情况下，迁移学习的效果不如完全重新训练。但是迁移学习所需要的训练时间和训练样本数要远远小于训练完整的模型。在没有GPU的普通台式机或者笔记本电脑上，6.5.2节中给出的TensorFlow训练过程只需要大约3小时，而且可以达到大概90%的正确率。

**TensorFlow实现迁移学习实例**

这里使用到的数据集是[http://download.tensorflow.org/example_images/flower_photos.tgz](http://download.tensorflow.org/example_images/flower_photos.tgz)，解压后的文件夹包含5个子文件夹，每一个子文件夹的名称为一种花的名称，代表了不同的类别。平均每种花有734张图片，每一张图片都是RGB色彩模式的，大小也不相同。和之前的样例不同，在这一节中给出的程序将直接处理没有整理过的图像数据。

**1. 数据处理**：

In [1]:
import glob
import os.path
import numpy as np
import tensorflow as tf
from tensorflow.python.platform import gfile

# 原始输入数据的目录，这个目录下有5个子目录，每个子目录底下保存这属于该类别的所有图片。
INPUT_DATA = '../../datasets/flower_photos'
# 输出文件地址。我们将整理后的图片数据通过numpy的格式保存。
OUTPUT_FILE = '../../datasets/flower_processed_data.npy'

# 测试数据和验证数据比例。
VALIDATION_PERCENTAGE = 10
TEST_PERCENTAGE = 10


# 读取数据并将数据分割成训练数据、验证数据和测试数据。
def create_image_lists(sess, testing_percentage, validation_percentage):
    sub_dirs = [x[0] for x in os.walk(INPUT_DATA)]
    is_root_dir = True
    
    # 初始化各个数据集。
    training_images = []
    training_labels = []
    validation_images = []
    validation_labels = []
    testing_images = []
    testing_labels = []
    current_label = 0
    
    # 读取所有的子目录。
    for sub_dir in sub_dirs:
        if is_root_dir:        # sub_dirs第一个元素为根目录路径
            is_root_dir = False
            continue

        # 获取一个子目录中所有的图片文件。
        extensions = ['jpg']
        file_list = []
        dir_name = os.path.basename(sub_dir)    # 返回path最后的文件名，/或\结尾则返回空值
        for extension in extensions:
            file_glob = os.path.join(INPUT_DATA, dir_name, '*.' + extension)     #  ../../../datasets/flower_photos\daisy\*.jpg
            file_list.extend(glob.glob(file_glob))    # 匹配所有符合条件文件，并以list返回
        if not file_list: continue
        print("processing: %s. total %d images."% (dir_name, len(file_list)))
        
        i = 0
        # 处理图片数据。
        for file_name in file_list:
            i += 1
            # 读取并解析图片，将图片转化为299*299以方便inception-v3模型来处理。
            image_raw_data = gfile.FastGFile(file_name, 'rb').read()
            image = tf.image.decode_jpeg(image_raw_data)
            if image.dtype != tf.int32:
                image = tf.image.convert_image_dtype(image, dtype=tf.int32)
            image = tf.image.resize_images(image, [299, 299])
            image_value = sess.run(image)
            
            # 随机划分数据集。
            chance = np.random.randint(100)
            if chance < validation_percentage:
                validation_images.append(image_value)
                validation_labels.append(current_label)
            elif chance < (testing_percentage + validation_percentage):
                testing_images.append(image_value)
                testing_labels.append(current_label)
            else:
                training_images.append(image_value)
                training_labels.append(current_label)
            if i % 100 == 0:
                print(i, "images processed.")
        current_label += 1
    
    # 将训练数据随机打乱以获得更好的训练效果。
    state = np.random.get_state()
    np.random.shuffle(training_images)
    np.random.set_state(state)
    np.random.shuffle(training_labels)
    
    return np.asarray([training_images, training_labels,
                       validation_images, validation_labels,
                       testing_images, testing_labels])


# 运行数据处理过程
with tf.Session() as sess:
    processed_data = create_image_lists(sess, TEST_PERCENTAGE, VALIDATION_PERCENTAGE)
    # 通过numpy格式保存处理后的数据。
    np.save(OUTPUT_FILE, processed_data)

processing: daisy. total 300 images.
100 images processed.
200 images processed.
300 images processed.
processing: dandelion. total 300 images.
100 images processed.
200 images processed.
300 images processed.
processing: roses. total 300 images.
100 images processed.
200 images processed.
300 images processed.
processing: sunflowers. total 300 images.
100 images processed.
200 images processed.
300 images processed.
processing: tulips. total 300 images.
100 images processed.
200 images processed.
300 images processed.


**2. 迁移学习：**

运行以上代码可以将所有的图片数据划分为训练集、验证集和测试集，并且将原始jpg格式转化为inception-v3模型需要的299\*299\*3的数字矩阵（*由于本人太穷，原数据集全部使用电脑会MemError，这里取每类图片后300张作为运行数据集*）。

谷歌提供的训练好的inception-v3模型地址如下：[http://download.tensorflow.org/models/inception_v3_2016_08_28.tar.gz](http://download.tensorflow.org/models/inception_v3_2016_08_28.tar.gz)。模型源代码[在这](https://github.com/tensorflow/models/blob/master/research/slim/nets/inception_v3.py)。

当新的数据集和预训练好的模型准备好之后，就可以通过下面代码来完成迁移学习了：

In [1]:
import glob
import os.path
import numpy as np
import tensorflow as tf
from tensorflow.python.platform import gfile
import tensorflow.contrib.slim as slim
# 加载通过TensorFlow-Slim定义好的inception_v3模型。
import tensorflow.contrib.slim.python.slim.nets.inception_v3 as inception_v3

# 1. 定义训练过程中将要使用到的常量。
# 处理好之后的数据文件。
INPUT_DATA = '../../datasets/flower_processed_data.npy'
# 保存训练好的模型的路径。这里可以将使用新数据训练得到的模型保存下来，如果计算充足，
# 还可以在训练完最后的全连接层之后再训练所有的网络层，使模型更贴近新数据。
TRAIN_FILE = 'train_dir/model'
# 谷歌提供的训练好的模型文件地址。因为GitHub无法保存大于100M的文件，所以
# 在运行时需要先自行从Google下载inception_v3.ckpt文件。
CKPT_FILE = '../../datasets/inception_v3.ckpt'

# 定义训练中使用的参数。
LEARNING_RATE = 0.0001
STEPS = 300
BATCH = 32
N_CLASSES = 5

# 不需要从谷歌训练好的模型中加载的参数。这里就是最后的全连接层。这里给出的是参数的前缀
CHECKPOINT_EXCLUDE_SCOPES = 'InceptionV3/Logits,InceptionV3/AuxLogits'
# 需要训练的网络层参数明层，在fine-tuning的过程中就是最后的全联接层。这里给出的是参数的前缀
TRAINABLE_SCOPES = 'InceptionV3/Logits,InceptionV3/AuxLogit'


# 2. 获取所有需要从谷歌训练好的模型中加载的参数。
def get_tuned_variables():
    exclusions = [scope.strip() for scope in CHECKPOINT_EXCLUDE_SCOPES.split(',')]
    variables_to_restore = []
    
    # 枚举inception-v3模型中所有的参数，然后判断是否需要从加载列表中移除。
    for var in slim.get_model_variables():
        excluded = False
        for exclusion in exclusions:
            if var.op.name.startswith(exclusion):
                excluded = True
                break
        if not excluded:
            variables_to_restore.append(var)
    return variables_to_restore


# 3. 获取所有需要训练的变量列表。
def get_trainable_variables():    
    scopes = [scope.strip() for scope in TRAINABLE_SCOPES.split(',')]
    variables_to_train = []
    
    # 枚举所有需要训练的参数前缀，并通过这些前缀找到所有需要训练的参数。
    for scope in scopes:
        variables = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope)
        variables_to_train.extend(variables)
    return variables_to_train


# 4. 定义训练过程
def main():
    # 0. 加载预处理好的数据。
    processed_data = np.load(INPUT_DATA)
    training_images = processed_data[0]
    n_training_example = len(training_images)
    training_labels = processed_data[1]
    
    validation_images = processed_data[2]
    validation_labels = processed_data[3]
    
    testing_images = processed_data[4]
    testing_labels = processed_data[5]
    print("%d training examples, %d validation examples and %d testing examples." % (
        n_training_example, len(validation_labels), len(testing_labels)))

    # 1. 定义inception-v3的输入输出
    images = tf.placeholder(tf.float32, [None, 299, 299, 3], name='input_images')
    labels = tf.placeholder(tf.int64, [None], name='labels')
    
    # 2. 定义前向传播、损失函数、反向传播
    # 定义inception-v3模型。因为谷歌给出的只有模型参数取值，所以这里
    # 需要在这个代码中定义inception-v3的模型结构。虽然理论上需要区分训练和
    # 测试中使用到的模型，也就是说在测试时应该使用is_training=False，但是
    # 因为预先训练好的inception-v3模型中使用的batch normalization参数与
    # 新的数据会有出入，所以这里直接使用同一个模型来做测试。
    with slim.arg_scope(inception_v3.inception_v3_arg_scope()):
        logits, _ = inception_v3.inception_v3(
            images, num_classes=N_CLASSES, is_training=True)
    
    # 定义损失函数
    trainable_variables = get_trainable_variables()    # 自定义函数，获取需要训练的变量
    tf.losses.softmax_cross_entropy(
        tf.one_hot(labels, N_CLASSES), logits, weights=1.0)
    total_loss = tf.losses.get_total_loss()
    
    # 定义反向传播
    train_step = tf.train.RMSPropOptimizer(LEARNING_RATE).minimize(total_loss)
    
    # 计算正确率。
    with tf.name_scope('evaluation'):
        correct_prediction = tf.equal(tf.argmax(logits, 1), labels)
        evaluation_step = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
                
    # 定义加载Google训练好的Inception-v3模型的Saver。
    load_fn = slim.assign_from_checkpoint_fn(
      CKPT_FILE,
      get_tuned_variables(),                          # 自定义函数，获取需要加载的参数
      ignore_missing_vars=True)
    
    # 3. 建立会话，训练模型
    saver = tf.train.Saver()      # 定义保存新模型的Saver。
    with tf.Session() as sess:
        # 初始化没有加载进来的变量。注意这个过程要在加载模型之前，
        # 否则初始化过程会将已经加载好的变量重新赋值。
        init = tf.global_variables_initializer()
        sess.run(init)
        
        # 加载谷歌已经训练好的模型。
        print('Loading tuned variables from %s' % CKPT_FILE)
        load_fn(sess)
            
        start = 0
        end = BATCH
        for i in range(STEPS):
            # 运行训练过程，这里不会更新全部参数，只会更新指定部分的参数
            _, loss = sess.run([train_step, total_loss], feed_dict={
                images: training_images[start:end], 
                labels: training_labels[start:end]})

            # 保存并输出日志
            if i % 30 == 0 or i + 1 == STEPS:
                saver.save(sess, TRAIN_FILE, global_step=i)
                validation_accuracy = sess.run(evaluation_step, feed_dict={
                    images: validation_images, labels: validation_labels})
                print('Step %d: Training loss is %.1f Validation accuracy = %.1f%%' % (
                    i, loss, validation_accuracy * 100.0))
                         
            # 获取下一个batch
            start = end
            if start == n_training_example:
                start = 0
            end = start + BATCH
            if end > n_training_example:
                end = n_training_example
            
        # 在最后的测试数据上测试正确率。
        test_accuracy = sess.run(evaluation_step, feed_dict={
            images: testing_images, labels: testing_labels})
        print('Final test accuracy = %.1f%%' % (test_accuracy * 100))
        
        
if __name__ == '__main__':
    main()

1231 training examples, 128 validation examples and 141 testing examples.
Loading tuned variables from ../../../datasets/inception_v3.ckpt
INFO:tensorflow:Restoring parameters from ../../../datasets/inception_v3.ckpt
Step 0: Training loss is 1.9 Validation accuracy = 17.2%
Step 30: Training loss is 1.7 Validation accuracy = 21.9%
Step 60: Training loss is 1.6 Validation accuracy = 49.2%
Step 90: Training loss is 1.2 Validation accuracy = 79.7%
Step 120: Training loss is 0.6 Validation accuracy = 89.1%
Step 150: Training loss is 0.3 Validation accuracy = 93.0%
Step 180: Training loss is 0.2 Validation accuracy = 93.0%
Step 210: Training loss is 0.2 Validation accuracy = 93.0%
Step 240: Training loss is 0.2 Validation accuracy = 92.2%
Step 270: Training loss is 0.2 Validation accuracy = 92.2%
Step 299: Training loss is 0.2 Validation accuracy = 92.2%
Final test accuracy = 91.5%


可以看到，模型在新的数据集上能够很快收敛，并达到还不错的分类效果。（虽然我只取了1500/3670张图片，最后的accuracy也基本达到了书中全部数据的准确率91.9%，可见迁移学习的效果非常好！）